## Tratamiento de datos

In [108]:
import pandas as pd
import numpy as np

### Parquet

### CSV

In [109]:
indie_games_csv = pd.read_csv('../datasets/indie-games-developers.csv')
indie_games_csv.head()

,Developer,City,Autonomous area,Country,Notable games,Notes
0,11 bit studios,Warsaw,NaN,Poland,AnomalyThis War of MineFrostpunk,NaN
1,ACE Team,Santiago,Santiago,Chile,Zeno ClashRock of Ages,NaN
2,Akupara Games,Los Angeles,California,United States,Whispering Willows The Metronomicon: Slay the ...,NaN
3,Alec Holowka,Winnipeg,Manitoba,Canada,Crayon Physics DeluxeI'm O.K – A Murder Simula...,NaN
4,Alientrap,Saskatoon,Saskatchewan,Canada,CapsizedApotheon,NaN


In [110]:
# comprobar si hay valores nulos 
indie_games_csv.isnull().sum()

Developer            0
City                15
Autonomous area     93
Country              0
Notable games        8
Notes              189
dtype: int64

In [111]:
indie_games_csv = indie_games_csv.drop(columns=['Notes', 'Notable games', 'Autonomous area'])
indie_games_csv.dropna(inplace=True)
indie_games_csv.head()

,Developer,City,Country
0,11 bit studios,Warsaw,Poland
1,ACE Team,Santiago,Chile
2,Akupara Games,Los Angeles,United States
3,Alec Holowka,Winnipeg,Canada
4,Alientrap,Saskatoon,Canada


In [112]:
video_games_csv = pd.read_csv('../datasets/video-games-developers.csv')
video_games_csv = video_games_csv.drop(columns=['Notes', 'Notable games, series or franchises', 'Administrative division', 'Est.'])
video_games_csv.head()

,Developer,City,Country
0,0verflow,Tokyo,Japan
1,11 bit studios,Warsaw,Poland
2,1C Company,Moscow,Russia
3,1-Up Studio,Tokyo,Japan
4,2K Czech,Brno,Czech Republic


### Unión de los csv

In [113]:
# unir los dos dataframes

indie_games_csv = pd.concat([indie_games_csv, video_games_csv], ignore_index=True)
# ordenar por nombre
indie_games_csv.sort_values(by='Developer', inplace=True)
indie_games_csv.head(10)

,Developer,City,Country
185,0verflow,Tokyo,Japan
188,1-Up Studio,Tokyo,Japan
186,11 bit studios,Warsaw,Poland
0,11 bit studios,Warsaw,Poland
187,1C Company,Moscow,Russia
189,2K Czech,Brno,Czech Republic
190,2K Games,Novato,United States
191,2K Sports,Novato,United States
192,343 Industries,Redmond,United States
193,38 Studios,Providence,United States


Filtramos por Developer para eliminar datos redundantes

In [114]:
indie_games_csv = indie_games_csv.drop_duplicates(subset='Developer', keep='first')
indie_games_csv.head(10)

,Developer,City,Country
185,0verflow,Tokyo,Japan
188,1-Up Studio,Tokyo,Japan
186,11 bit studios,Warsaw,Poland
187,1C Company,Moscow,Russia
189,2K Czech,Brno,Czech Republic
190,2K Games,Novato,United States
191,2K Sports,Novato,United States
192,343 Industries,Redmond,United States
193,38 Studios,Providence,United States
194,3D Realms,Garland,United States
